# First-order derivatives
Here, we derive an $\mathcal{O}(h^4)$ finite-difference approximation for the first derivative of a function, $f : U \to \mathbb{R}$, for some open set $U \subseteq \mathbb{R}$.
We will assume that $f$ is at least $5$ times differentiable.

Let us begin with a forward finite difference method. Let $x \in U$. We seek coefficients $c_0, c_1, c_2, c_3, c_4$ such that 

$$\frac{c_0 f(x_0) + c_1 f(x + h) + c_2 f(x + 2h) + c_3 f(x + 3h) + c_4 f(x + 4h)}{h} = f'(x) + \mathcal{O}(h^4)$$

We will now take $5$'th order taylor expansions of $f(x + h), f(x + 2h), f(x + 3h)$, and $f(x + 4h)$ and find coefficients which make the expression above true.
Since $f$ is $5$ times differentiable,  

$$
\begin{aligned} 
f(x + h) &= f(x)    + h f'(x)       &&+ \frac{h^2}{2} f''(x)      &&+ \frac{h^3}{6} f^{(3)}(x)      &&+ \frac{h^4}{24} f^{(4)}(x)     &&+ \mathcal{O}(h^5) \\
f(x + 2h) &= f(x)   + 2 h f'(x)     &&+ 2h^2 f''(x)               &&+ \frac{4 h^3}{3} f^{(3)}(x)    &&+ \frac{2 h^4}{3} f^{(4)}(x)    &&+ \mathcal{O}(h^5) \\
f(x + 3h) &= f(x)   + 3 h f'(x)     &&+ \frac{9 h^2}{2} f''(x)    &&+ \frac{9 h^3}{2} f^{(3)}(x)    &&+ \frac{27 h^4}{8} f^{(4)}(x)   &&+ \mathcal{O}(h^5) \\
f(x + 4h) &= f(x)   + 4 h f'(x)     &&+ 8 h^2 f''(x)              &&+ \frac{32 h^3}{3} f^{(3)}(x)   &&+ \frac{32 h^4}{3} f^{(4)}(x)   &&+ \mathcal{O}(h^5) \\
\end{aligned}
$$

Thus, we must have 

$$
\begin{aligned} 
c_0 f(x_0) + c_1 f(x + h) + c_2 f(x + 2h) + c_3 f(x + 3h) + c_4 f(x + 4h) = 
&               \left( c_0                  + c_1               + c_2               + c_3                   + c_4                   \right) f(x) \\
+ &h            \left( c_1                  + 2 c_2             + 3 c_3             + 4 c_4                 \right) f'(x)\\
+ &h^2          \left( \frac{c_1}{2}        + 2 c_2             + \frac{9 c_3}{2}   + 8 c_4                 \right) f''(x) \\
+ &h^3          \left( \frac{c_1}{6}        + \frac{4 c_2}{3}   + \frac{9 c_3}{2}   + \frac{32 c_4}{3}      \right) f^{(3)}(x) \\
+ &h^4          \left( \frac{c_1}{24}       + \frac{2 c_2}{3}   + \frac{27 c_3}{8}  + \frac{32 c_4}{3}      \right)f^{(4)}(x) \\
\end{aligned}
$$

For this to equal $f'(x) h + \mathcal{O}(h^5)$, we need the following system of equations to hold:

$$
\begin{bmatrix}
1, &&1,             &&1,                &&1,                &&1            \\
0, &&1,             &&2,                &&3,                &&4            \\
0, &&\frac{1}{2},   &&2,                &&\frac{9}{2},      &&8            \\
0, &&\frac{1}{6},   &&\frac{4}{3},      &&\frac{9}{2},      &&\frac{32}{3} \\
0, &&\frac{1}{24},  &&\frac{2}{3},      &&\frac{27}{8},     &&\frac{32}{3}
\end{bmatrix}
\begin{bmatrix}
c_0 \\
c_1 \\
c_2 \\
c_3 \\
c_4 
\end{bmatrix}
= 
\begin{bmatrix}
0 \\
1 \\
0 \\
0 \\
0
\end{bmatrix}
$$

In [21]:
import numpy;

# Set up an array to hold the matrix of coefficients.
A = numpy.empty((5, 5), dtype = numpy.float32);
A[0, :] = [1,   1,      1,      1,      1];
A[1, :] = [0,   1,      2,      3,      4];
A[2, :] = [0,   1/2,    2,      9/2,    8];
A[3, :] = [0,   1/6,    4/3,    9/2,    32/3];
A[4, :] = [0,   1/24,   2/3,    27/8,   32/3];

# Set up a vector to hold the right hand side
b = numpy.array((0, 1, 0, 0, 0), dtype = numpy.float32);

# Solve for the coefficients!
c = numpy.linalg.solve(A, b);

# Now report the results.
print("c_0 = %g" % c[0]);
print("c_1 = %g" % c[1]);
print("c_2 = %g" % c[2]);
print("c_3 = %g" % c[3]);
print("c_4 = %g" % c[4]);

c_0 = -2.08333
c_1 = 4
c_2 = -3
c_3 = 1.33333
c_4 = -0.25


Now let's move onto an approximation that uses one step from the past and three in the future. 
Specifically, we seek coefficients $c_{-1}, c_0, c_1, c_2, c_3$ such that

$$\frac{c_{-1} f(x - h) + c_0 f(x) + c_1 f(x + h) + c_2 f(x + 2h) + c_3 f(x + 3h)}{h} = f'(x) + \mathcal{O}(h^4).$$

As before, we will take taylor expansions of $f(x - h), f(x + h), f(x + 2h)$, and $f(x + 3h)$ and then findinging coefficients which make the expression above true.
Thankfully, we can reuse a lot of work from the forward-difference case.
Specifically, the only new term is 

$$f(x - h) = f(x) - h f'(x) + \frac{h^2}{2} f''(x) - \frac{h^3}{6} f^{(3)}(x) + \frac{h^4}{24} f^{(4)}(x) + \mathcal{O}(h^5).$$

Repeating the same analysis as before, we can conclude that if we want an $\mathcal{O}(h^4)$ approximation, then the coefficients must satisfy 

$$
\begin{bmatrix}
1,              &&1,    &&1,             &&1,                &&1            \\
-1,             &&0,    &&1,             &&2,                &&3            \\
\frac{1}{2},    &&0,    &&\frac{1}{2},   &&2,                &&\frac{9}{2}  \\
-\frac{1}{6},   &&0,    &&\frac{1}{6},   &&\frac{4}{3},      &&\frac{9}{2}  \\
\frac{1}{24},   &&0,    &&\frac{1}{24},  &&\frac{2}{3},      &&\frac{27}{8} 
\end{bmatrix}
\begin{bmatrix}
c_{-1} \\
c_0 \\
c_1 \\
c_2 \\
c_3 
\end{bmatrix}
= 
\begin{bmatrix}
0 \\
1 \\
0 \\
0 \\
0 
\end{bmatrix}
$$

In [29]:
import numpy;

# Set up an array to hold the matrix of coefficients.
A = numpy.empty((5, 5), dtype = numpy.float32);
A[0, :] = [1,       1,  1,      1,      1];
A[1, :] = [-1,      0,  1,      2,      3];
A[2, :] = [1/2,     0,  1/2,    2,      9/2];
A[3, :] = [-1/6,    0,  1/6,    4/3,    9/2];
A[4, :] = [1/24,    0,  1/24,   2/3,    27/8];

# Set up a vector to hold the right hand side
b = numpy.array((0, 1, 0, 0, 0), dtype = numpy.float32);

# Solve for the coefficients!
c = numpy.linalg.solve(A, b);

# Now report the results.
print("c_{-1}   = %g" % c[0]);
print("c_0      = %g" % c[1]);
print("c_1      = %g" % c[2]);
print("c_2      = %g" % c[3]);
print("c_3      = %g" % c[4]);

c_{-1}   = -0.25
c_0      = -0.833333
c_1      = 1.5
c_2      = -0.5
c_3      = 0.0833333


Finally, let us derive an $\mathcal{O}(h^4)$ central difference scheme. 
We seek coefficients $c_{-2}, c_{-1}, c_0, c_1$, and $c_2$ such that

$$\frac{c_{-2} f(x - 2h) + c_{-1} f(x - h) + c_0 f(x) + c_1 f(x + h) + c_2 f(x + 2h)}{h} = f'(x) + \mathcal{O}(h^4).$$

Once again, we will take a taylor expansion of $f(x - 2h), f(x - h), f(x + h)$, and $f(x + 2h)$ to find the coefficients.
We already know the form of the expansion for $f(x - h), f(x + h)$, and $f(x + 2h)$.
Thus, $f(x - 2h)$ is the only new term. 
We know that 

$$f(x - 2h) = f(x) - 2h f'(X) + 2 h^2 f''(x) - \frac{4 h^3}{3} f^{(3)}(x) + \frac{2 h^4}{3} f^{(4)}(x) + \mathcal{O}(h^5).$$

Repeating the same analysis as before, we can conclude that $c_{-2}, c_{-1}, c_0, c_1,$ and $c_2$ must satisfy the following linear system

$$
\begin{bmatrix}
1,              &&1,              &&1,    &&1,             &&1              \\
-2,             &&-1,             &&0,    &&1,             &&2              \\
2,              &&\frac{1}{2},    &&0,    &&\frac{1}{2},   &&2              \\
-\frac{4}{3},   &&-\frac{1}{6},   &&0,    &&\frac{1}{6},   &&\frac{4}{3}    \\
\frac{2}{3},    &&\frac{1}{24},   &&0,    &&\frac{1}{24},  &&\frac{2}{3}
\end{bmatrix}
\begin{bmatrix}
c_{-2} \\
c_{-1} \\
c_0 \\
c_1 \\
c_2 
\end{bmatrix}
= 
\begin{bmatrix}
0 \\
1 \\
0 \\
0 \\
0 
\end{bmatrix}
$$

In [30]:
import numpy;

# Set up an array to hold the matrix of coefficients.
A = numpy.empty((5, 5), dtype = numpy.float32);
A[0, :] = [1,       1,       1,  1,      1];
A[1, :] = [-2,      -1,      0,  1,      2];
A[2, :] = [2,       1/2,     0,  1/2,    2];
A[3, :] = [-4/3,    -1/6,    0,  1/6,    4/3];
A[4, :] = [2/3,     1/24,    0,  1/24,   2/3];

# Set up a vector to hold the right hand side
b = numpy.array((0, 1, 0, 0, 0), dtype = numpy.float32);

# Solve for the coefficients!
c = numpy.linalg.solve(A, b);

# Now report the results.
print("c_{-2}   = %g" % c[0]);
print("c_{-1}   = %g" % c[1]);
print("c_0      = %g" % c[2]);
print("c_1      = %g" % c[3]);
print("c_2      = %g" % c[4]);

c_{-2}   = 0.0833333
c_{-1}   = -0.666667
c_0      = 0
c_1      = 0.666667
c_2      = -0.0833333


# Second-order derivatives
Here, we derive an $\mathcal{O}(h^4)$ finite-difference approximation for the second derivative of a function, $f : U \to \mathbb{R}$, for some open set $U \subseteq \mathbb{R}$.
We will assume that $f$ is at least $6$ times differentiable. 

Let us begin with a forward finite difference method. 
Let $x \in U$.
Our goal is to find coefficients $c_0, \ldots, c_5$ such that

$$\frac{c_0 f(x_0) + c_1 f(x + h) + c_2 f(x + 2h) + c_3 f(x + 3h) + c_4 f(x + 4h) + c_5 f(x + 5h)}{h^2} = f''(x) + \mathcal{O}(h^4).$$

We will now take $5$'th order taylor expansions of $f(x + h), f(x + 2h), f(x + 3h), f(x + 4h)$, and $f(x + 5h)$ and find coefficients which make the expression above true.
Specifically, since $f$ is $6$ times differentiable, 

$$
\begin{aligned} 
f(x + h) &= f(x)    + h f'(x)       &&+ \frac{h^2}{2} f''(x)      &&+ \frac{h^3}{6} f^{(3)}(x)      &&+ \frac{h^4}{24} f^{(4)}(x)     &&+ \frac{h^5}{120} f^{(5)}(x)    &&+ \mathcal{O}(h^6) \\
f(x + 2h) &= f(x)   + 2 h f'(x)     &&+ 2h^2 f''(x)               &&+ \frac{4 h^3}{3} f^{(3)}(x)    &&+ \frac{2 h^4}{3} f^{(4)}(x)    &&+ \frac{4 h^5}{15} f^{(5)}(x)   &&+ \mathcal{O}(h^6) \\
f(x + 3h) &= f(x)   + 3 h f'(x)     &&+ \frac{9 h^2}{2} f''(x)    &&+ \frac{9 h^3}{2} f^{(3)}(x)    &&+ \frac{27 h^4}{8} f^{(4)}(x)   &&+ \frac{81 h^5}{40} f^{(5)}(x)  &&+ \mathcal{O}(h^6) \\
f(x + 4h) &= f(x)   + 4 h f'(x)     &&+ 8 h^2 f''(x)              &&+ \frac{32 h^3}{3} f^{(3)}(x)   &&+ \frac{32 h^4}{3} f^{(4)}(x)   &&+ \frac{128 h^5}{15} f^{(5)}(x) &&+ \mathcal{O}(h^6) \\
f(x + 5h) &= f(x)   + 5 h f'(x)     &&+ \frac{25 h^2}{2} f''(x)   &&+ \frac{125 h^3}{6} f^{(3)}(x)  &&+ \frac{625 h^4}{24} f^{(4)}(x) &&+ \frac{625 h^5}{24} f^{(5)}(x) &&+ \mathcal{O}(h^6).
\end{aligned}
$$

Thus, we must have 

$$
\begin{aligned} 
c_0 f(x_0) + c_1 f(x + h) + c_2 f(x + 2h) + c_3 f(x + 3h) + c_4 f(x + 4h) + c_5 f(x + 5h) = 
&               \left( c_0              + c_1               + c_2               + c_3                   + c_4                   + c_5             \right) f(x) \\
+ &h            \left( c_1              + 2 c_2             + 3 c_3             + 4 c_4                 + 5 c_5                 \right) f'(x)\\
+ &h^2          \left( \frac{c_1}{2}    + 2 c_2             + \frac{9 c_3}{2}   + 8 c_4                 + \frac{25 c_5}{2}      \right) f''(x) \\
+ &h^3          \left( \frac{c_1}{6}    + \frac{4 c_2}{3}   + \frac{9 c_3}{2}   + \frac{32 c_4}{3}      + \frac{125 c_5}{6}     \right) f^{(3)}(x) \\
+ &h^4          \left( \frac{c_1}{24}   + \frac{2 c_2}{3}   + \frac{27 c_3}{8}  + \frac{32 c_4}{3}      + \frac{625 c_5}{24}    \right)f^{(4)}(x) \\
+ &h^5          \left( \frac{c_1}{120}  + \frac{4 c_2}{15}  + \frac{81 c_3}{40} + \frac{128 c_4}{15}    + \frac{625 c_5}{24}    \right) f^{(5)}(x) + \mathcal{O}(h^6)
\end{aligned}
$$

For this to equal $f''(x) h^2 + \mathcal{O}(h^6)$, we need the following system of equations to hold:

$$
\begin{bmatrix}
1, &&1,             &&1,                &&1,                &&1,                &&1 \\
0, &&1,             &&2,                &&3,                &&4,                &&5 \\
0, &&\frac{1}{2},   &&2,                &&\frac{9}{2},      &&8,                &&\frac{25}{2} \\
0, &&\frac{1}{6},   &&\frac{4}{3},      &&\frac{9}{2},      &&\frac{32}{3},     &&\frac{125}{6} \\
0, &&\frac{1}{24},  &&\frac{2}{3},      &&\frac{27}{8},     &&\frac{32}{3},     &&\frac{625}{24} \\
0, &&\frac{1}{120}, &&\frac{4}{15},     &&\frac{81}{40},    &&\frac{128}{15},   &&\frac{625}{24} 
\end{bmatrix}
\begin{bmatrix}
c_0 \\
c_1 \\
c_2 \\
c_3 \\
c_4 \\
c_5
\end{bmatrix}
= 
\begin{bmatrix}
0 \\
0 \\
1 \\
0 \\
0 \\
0
\end{bmatrix}
$$

In [28]:
import numpy;

# Set up an array to hold the matrix of coefficients.
A = numpy.empty((6, 6), dtype = numpy.float32);
A[0, :] = [1,   1,      1,      1,      1,      1];
A[1, :] = [0,   1,      2,      3,      4,      5];
A[2, :] = [0,   1/2,    2,      9/2,    8,      25/2];
A[3, :] = [0,   1/6,    4/3,    9/2,    32/3,   125/6];
A[4, :] = [0,   1/24,   2/3,    27/8,   32/3,   625/24];
A[5, :] = [0,   1/120,  4/15,   81/40,  128/15, 625/24];

# Set up a vector to hold the right hand side.
b = numpy.array((0, 0, 1, 0, 0, 0), dtype = numpy.float32);

# Solve!
c = numpy.linalg.solve(A, b);

# Report results.
print("c_0 = %g" % c[0]);
print("c_1 = %g" % c[1]);
print("c_2 = %g" % c[2]);
print("c_3 = %g" % c[3]);
print("c_4 = %g" % c[4]);
print("c_5 = %g" % c[5]);

c_0 = 3.75
c_1 = -12.8333
c_2 = 17.8333
c_3 = -13
c_4 = 5.08333
c_5 = -0.833333


We can now move onto an approximation that uses one step of the past and four in the future. 
That is, we want to find coefficients $c_{-1}, c_0, c_1, c_2, c_3, c_4$ such that 

$$ \frac{c_{-1} f(x - h) + c_0 f(x) + c_1 f(x + h) + c_2 f(x + 2h) + c_3 f(x + 3h) + c_4 f(x + 4h)}{h^2} = f''(x) + \mathcal{O}(h^4).$$

As in the last case, we take a taylor expansion of this expression.
We can reuse a lot of the work from the previous step. 
Specifically, the only new term is 

$$f(x - h) = f(x) - h f'(x) + \frac{h^2}{2} f''(x) - \frac{h^3}{6} f^{(3)}(x) + \frac{h^4}{24} f^{(4)}(x) - \frac{h^5}{120} f^{(5)}(x) + \mathcal{O}(h^6).$$

Repeating the same analysis as before, we can conclude that if we want an $\mathcal{O}(h^4)$ approximation, then the coefficients must satisfy 

$$
\begin{bmatrix}
1,              &&1,    &&1,             &&1,                &&1,                &&1                \\
-1,             &&0,    &&1,             &&2,                &&3,                &&4                \\
\frac{1}{2},    &&0,    &&\frac{1}{2},   &&2,                &&\frac{9}{2},      &&8                \\
-\frac{1}{6},   &&0,    &&\frac{1}{6},   &&\frac{4}{3},      &&\frac{9}{2},      &&\frac{32}{3}     \\
\frac{1}{24},   &&0,    &&\frac{1}{24},  &&\frac{2}{3},      &&\frac{27}{8},     &&\frac{32}{3}     \\
-\frac{1}{120}, &&0,    &&\frac{1}{120}, &&\frac{4}{15},     &&\frac{81}{40},    &&\frac{128}{15}
\end{bmatrix}
\begin{bmatrix}
c_{-1} \\
c_0 \\
c_1 \\
c_2 \\
c_3 \\
c_4
\end{bmatrix}
= 
\begin{bmatrix}
0 \\
0 \\
1 \\
0 \\
0 \\
0
\end{bmatrix}



In [24]:
import numpy; 

# Set up an array to hold the matrix of coefficients.
A = numpy.empty((6, 6), dtype = numpy.float32);
A[0, :] = [1,       1,   1,      1,      1,      1];
A[1, :] = [-1,      0,   1,      2,      3,      4];
A[2, :] = [1/2,     0,   1/2,    2,      9/2,    8];
A[3, :] = [-1/6,    0,   1/6,    4/3,    9/2,    32/3];
A[4, :] = [1/24,    0,   1/24,   2/3,    27/8,   32/3];
A[5, :] = [-1/120,  0,   1/120,  4/15,   81/40,  128/15];

# Set up a vector to hold the right hand side.
b = numpy.array((0, 0, 1, 0, 0, 0), dtype = numpy.float32);

# Solve!
c = numpy.linalg.solve(A, b);

# Report results.
print("c_{-1} = %g" % c[0]);
print("c_0 = %g" % c[1]);
print("c_1 = %g" % c[2]);
print("c_2 = %g" % c[3]);
print("c_3 = %g" % c[4]);
print("c_4 = %g" % c[5]);

c_{-1} = 0.833333
c_0 = -1.25
c_1 = -0.333334
c_2 = 1.16667
c_3 = -0.5
c_4 = 0.0833334


Finally, we can move onto the central difference approximation. 
For this, we want to find coefficients $c_{-2}, c_{-1}, c_0, c_1, c_2$ such that

$$ \frac{c_{-2} f(x - 2h) + c_{-1} f(x - h) + c_0 f(x) + c_1 f(x + h) + c_2 f(x + 2h)}{h^2} = f''(x) + \mathcal{O}(h^4)$$

As in the last case, we take a taylor expansion of this expression.
Once again, we can reuse a lot of the work from the previous step. 
Specifically, the only new term is 

$$f(x - 2h) = f(x) - 2h f'(x) + 2h^2 f''(x) - \frac{4 h^3}{3} f^{(3)}(x) + \frac{2 h^4}{3} f^{(4)}(x) - \frac{4 h^5}{15} f^{(5)}(x) + \mathcal{O}(h^6).$$

Repeating the same analysis as before, we can conclude that if we want an $\mathcal{O}(h^4)$ approximation, then the coefficients must satisfy 

$$
\begin{bmatrix}
1,              &&1,              &&1,    &&1,             &&1              \\
-2,             &&-1,             &&0,    &&1,             &&2              \\
2,              &&\frac{1}{2},    &&0,    &&\frac{1}{2},   &&2              \\
-\frac{4}{3},   &&-\frac{1}{6},   &&0,    &&\frac{1}{6},   &&\frac{4}{3}    \\
\frac{2}{3},    &&\frac{1}{24},   &&0,    &&\frac{1}{24},  &&\frac{2}{3}    \\
-\frac{4}{15},  &&-\frac{1}{120}, &&0,    &&\frac{1}{120}, &&\frac{4}{15}
\end{bmatrix}
\begin{bmatrix}
c_{-2} \\
c_{-1} \\
c_0 \\
c_1 \\
c_2
\end{bmatrix}
= 
\begin{bmatrix}
0 \\
0 \\
1 \\
0 \\
0 \\
0
\end{bmatrix}
$$

Notice that the last row of this matrix is simply $-1/30$ of the end row plus $1/4$ times the 4th. 
Thus, we can instead solve the following reduced system (symmetry is our friend):

$$
\begin{bmatrix}
1,              &&1,              &&1,    &&1,             &&1              \\
-2,             &&-1,             &&0,    &&1,             &&2              \\
2,              &&\frac{1}{2},    &&0,    &&\frac{1}{2},   &&2              \\
-\frac{4}{3},   &&-\frac{1}{6},   &&0,    &&\frac{1}{6},   &&\frac{4}{3}    \\
\frac{2}{3},    &&\frac{1}{24},   &&0,    &&\frac{1}{24},  &&\frac{2}{3}
\end{bmatrix}
\begin{bmatrix}
c_{-2} \\
c_{-1} \\
c_0 \\
c_1 \\
c_2
\end{bmatrix}
= 
\begin{bmatrix}
0 \\
0 \\
1 \\
0 \\
0
\end{bmatrix}
$$

In [25]:
import numpy; 

# Set up an array to hold the matrix of coefficients.
A = numpy.empty((5, 5), dtype = numpy.float32);
A[0, :] = [1,       1,       1,   1,      1];
A[1, :] = [-2,      -1,      0,   1,      2];
A[2, :] = [2,       1/2,     0,   1/2,    2];
A[3, :] = [-4/3,    -1/6,    0,   1/6,    4/3];
A[4, :] = [2/3,     1/24,    0,   1/24,   2/3];

# Set up a vector to hold the right hand side.
b = numpy.array((0, 0, 1, 0, 0), dtype = numpy.float32);

# Solve!
c = numpy.linalg.solve(A, b);

# Report results.
print("c_{-2} = %g" % c[0]);
print("c_{-1} = %g" % c[1]);
print("c_0 = %g" % c[2]);
print("c_1 = %g" % c[3]);
print("c_2 = %g" % c[4]);

c_{-2} = -0.0833333
c_{-1} = 1.33333
c_0 = -2.5
c_1 = 1.33333
c_2 = -0.0833333
